In [ ]:

%glue_version 4.0
%additional_python_modules tgedr-nihao
# --- nn
# %region eu-central-1
# %iam_role arn:aws:iam::988864172292:role/custom_role_for_glue_interactive_sessions
# --- tgedr
%region eu-north-1
%iam_role arn:aws:iam::041651352119:role/tgedr_it_role_for_glue_service




In [ ]:
%%configure
{
  "enable_spark_ui": "true",
  "python_version": "3.10"
}

In [ ]:
spark

In [ ]:
import sys
import logging

from pyspark.context import SparkContext
from pyspark.sql import DataFrame

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job

from tgedr.nihao.processors.ticker_simple_analysis import TickerSimpleAnalysis

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

sys.argv+=['--source_bucket_key', "s3://de-landing-dev-c44cd1c4-19d5-4b81-af9c-37cb1abac051/dataengineering"]
sys.argv+=['--target_bucket_key', "s3://de-bronze-dev-ae67b8d7-3e23-4c66-a101-33a096537444/dataengineering"]
sys.argv+=['--job_name', 'my_test_job']

args = getResolvedOptions(
    sys.argv, ["source_bucket_key", "target_bucket_key", "job_name"]
)
#args = {"source_bucket_key": "s3://de-landing-dev-c44cd1c4-19d5-4b81-af9c-37cb1abac051/dataengineering", 
#        "target_bucket_key": "s3://de-bronze-dev-ae67b8d7-3e23-4c66-a101-33a096537444/dataengineering", 
#        "job_name": "tgedr_test"}
logger.info(f"[in] args: {args}")

In [ ]:
# get glue context
glueContext = GlueContext(SparkContext.getOrCreate())

source_bucket_key = args["source_bucket_key"]
target_bucket_key = args["target_bucket_key"]
job_name = args["job_name"]

In [ ]:
job = Job(glueContext)
job.init(job_name, args)

In [ ]:
input = glueContext.spark_session.read.parquet(source_bucket_key)
result: DataFrame = TickerSimpleAnalysis().process(input)
    

In [ ]:
result.show()

In [ ]:
result.write.parquet(target_bucket_key)
job.commit()

In [ ]:
# parquet_file = "s3://988864172292-gscdm-dev-eu-central-1-landing-zone/dh_cdm/events"
parquet_file = "s3://de-landing-dev-c44cd1c4-19d5-4b81-af9c-37cb1abac051/dataengineering"

In [ ]:
df = spark.read.parquet(parquet_file)